In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *

races = ['white', 'black', 'hisp']
parties = ['red', 'blue']

def prep_data(level='cntyvtd', election='Pres_20'):
    tbl = f'cmat-315920.VRA.{level}'
    df = run_query(f'select * from {tbl}')
    geo = gpd.GeoSeries.from_wkt(df.pop('polygon'), crs=crs_census).simplify(0.003).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
    cols = ['county', 'vap_pop'] + [f'vap_{r}' for r in races] + [f'{election}_votes', f'{election}_red_pct']
    if level == 'cntyvtd':
        cols = ['cntyvtd'] + cols
    df = df[cols]
    df[f'{election}_blue_pct'] = 100 - df[f'{election}_red_pct']
    for party in parties:
        df[f'{election}_{party}_votes'] = df[f'{election}_votes'] * df[f'{election}_{party}_pct'] / 100
    df[f'{election}_vote_rate'] = df[f'{election}_votes'] / df['vap_pop']
    for race in races:
        df[f'{election}_{race}_votes'] = df[f'vap_{race}'] * df[f'{election}_vote_rate']
    return df, geo

def hisp_red_pct(df, white_red_pct=100, black_red_pct=10):
    a = [x for x in df.columns if 'votes' in x][0]
    election = a[:a.rfind('_')]
    df[f'{election}_white_red_votes'] = df[f'{election}_white_votes'] * white_red_pct / 100
    df[f'{election}_black_red_votes'] = df[f'{election}_black_votes'] * black_red_pct / 100
    df[f'{election}_hisp_red_votes' ] = df[f'{election}_red_votes'] - df[f'{election}_white_red_votes'] - df[f'{election}_black_red_votes']
    df[f'{election}_white_red_pct']   = white_red_pct
    df[f'{election}_black_red_pct']   = black_red_pct
    df[f'{election}_hisp_red_pct']    = np.clip(df[f'{election}_hisp_red_votes'] / df[f'{election}_hisp_votes'], 0, 1) * 100
    return df

level    = 'county'
election = 'Pres_20'
wr = f'{election}_white_red_pct'
hr = f'{election}_hisp_red_pct'
br = f'{election}_black_red_pct'
data, geo = prep_data(level=level, election=election)

In [ ]:
import pathlib, numpy as np, pandas as pd

L = list()
for b in range(0, 101, 5):
    for w in range(0, 101, 5):
        X = hisp_red_pct(data.copy(), black_red_pct=b, white_red_pct=w)[['county', hr]]
        X.insert(1, wr, w)
        X.insert(1, br, b)
        L.append(X)
df = pd.concat(L).reset_index()

path = pathlib.Path('/home/jupyter/VRA')
datafile = path / 'hisp_red_min.csv'
geofile  = path / 'TX.geojson'

datafile.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(datafile, index=False)
with open(geofile, 'w') as f:
    f.write(geo.to_json())

In [ ]:
import pathlib, numpy as np, pandas as pd, json, urllib, plotly.express as px
path = pathlib.Path('/content')
geofile  = path / 'TX.geojson'
datafile = path / 'hisp_red_min.csv'
figfile  = datafile.with_suffix('.html')
datafile.parent.mkdir(parents=True, exist_ok=True)

def get(file, url):
    if not file.is_file():
        url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
        print(f'downloading {file.name} from {url}')
        urllib.request.urlretrieve(url, file)
get(datafile, 'https://drive.google.com/file/d/1xTo59uoSvUUKMr_LOlS5qJu9L9sU77Hz/view?usp=sharing')
get(geofile , 'https://drive.google.com/file/d/1FRjVmuzyQDGlTzhpC9xSRlNwt4cgCVgb/view?usp=sharing')

df = pd.read_csv(datafile)
with open(geofile, 'r') as f:
    geojson = json.load(f)

br = [x for x in df.columns if 'black' in x][0]
wr = [x for x in df.columns if 'white' in x][0]
hr = [x for x in df.columns if 'hisp'  in x][0]
election = hr.split('_hisp')[0]
    
x_range = [-106.645646, -93.508292]
y_range = [25.837377, 36.500704]
aspect = (x_range[1]-x_range[0])/(y_range[1]-y_range[0])
height = 500
width = height * aspect
lon = (x_range[1]+x_range[0])/2
lat = (y_range[1]+y_range[0])/2
cmap = 'bluered'
cmap = 'jet'

fig = px.choropleth_mapbox(df, geojson=geojson, locations='index',
                           color = hr,
                           color_continuous_scale=cmap,
                           range_color = (0, 100),
                           animation_frame = br,
                           hover_name = 'county',
                           hover_data = {hr:':.1f', br:False, wr:False, 'index':False},
                           mapbox_style="carto-positron",
                           zoom=3.8, 
                           center = {"lat": lat, "lon": lon},
                           width=width,
                           height=height,
                           opacity=0.5,
                        #    labels={hr:'Hispanic red pct min', br:'Black red pct est'}
                          )
fig.update_layout(title_text=election, title_x=0.5, title_y=0.92)
with open(figfile, 'w') as f:
    f.write(fig.to_html(auto_play=False))
fig

In [ ]:
import pathlib, numpy as np, pandas as pd, json, plotly.express as px
path = pathlib.Path('/home/jupyter/VRA')

geofile  = path / 'TX.geojson'
datafile = path / 'hisp_red_min.csv'
figfile  = datafile.with_suffix('.html')
df = pd.read_csv(datafile)
with open(geofile, 'r') as f:
    geojson = json.load(f)

br = [x for x in df.columns if 'black' in x][0]
wr = [x for x in df.columns if 'white' in x][0]
hr = [x for x in df.columns if 'hisp'  in x][0]
election = hr.split('_hisp')[0]
    
x_range = [-106.645646, -93.508292]
y_range = [25.837377, 36.500704]
aspect = (x_range[1]-x_range[0])/(y_range[1]-y_range[0])
height = 500
width = height * aspect
lon = (x_range[1]+x_range[0])/2
lat = (y_range[1]+y_range[0])/2
cmap = 'bluered'
cmap = 'jet'

fig = px.choropleth_mapbox(df.reset_index(), geojson=geojson, locations='index',
                           color = hr,
                           color_continuous_scale=cmap,
                           range_color = (0, 100),
                           animation_frame = br,
                           hover_name = 'county',
                           hover_data = {hr:':.1f', br:False, wr:False, 'index':False},
                           mapbox_style="carto-positron",
                           zoom=3.8, 
                           center = {"lat": lat, "lon": lon},
                           width=width,
                           height=height,
                           opacity=0.5,
                           labels={hr:'Hispanic red pct min', br:'Black red pct est'}
                          )
fig.update_layout(title_text=election, title_x=0.5, title_y=0.92)
with open(figfile, 'w') as f:
    f.write(fig.to_html(auto_play=False))
fig

In [ ]:
for level in ['county', 'cntyvtd']:    
    df = prep_data(level=level)
    df = hisp_red_pct(df)
    # df.to_csv(f'{path}/VRA_min_hisp_red_pct_{level}.csv')
df.head(3)

In [ ]:
###### Build VRA statistical models ######
###### This cell defines, but does not run, the modeling functions. ######
###### It does not produce any output, so it might not seem to do anything. ######
###### You will specifty model parameters and run these functions later. ######

import numpy as np, pandas as pd, statsmodels.api as sm
# I will try to read VRA csv files from the local path below.
# If that fails, I will try the google drive URL in model_votes.
path = f'/home/jupyter/redistricting_data/'

###### Helper functions ######
def listify(x):
    """ensure x is a list"""
    if x is None:
        x = []
    elif isinstance(x, str):
        x = [x]
    return x

def check(x, valid):
    """check x is a valid value"""
    bad = set(listify(x)).difference(valid)
    assert len(bad)==0,  f'unknown {bad} ... must be one of {valid}'

def read_data(level):
    valid_levels = {'county', 'cntyvtd'}
    check(level, valid_levels)
    try:
        file = path + f'vra_{level}.csv'
        df = pd.read_csv(file)
        print(f'using local {file}')
    except:
        if level == 'county':
            url = 'https://drive.google.com/file/d/143OH38F_fTqSnwwTBkkGsniBSeo8by18/view?usp=sharing'
        else:
            url = 'https://drive.google.com/file/d/149IB9m4YKcgrleAJTd44yJiu-7pHh0-n/view?usp=sharing'
        url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
        df = pd.read_csv(url)
        print(f'using remote {url}')
    return df
    
    
###### Define function that builds weighted least squares model  ######
def model_votes(level, metric, election, races, predictors=None, interactions=None, const=True, weight='vap_pop'):
    df = read_data(level)
    valid_elections = {x[:-8] for x in df.columns if 'red_pct' in x}
    valid_metrics   = {'red_pct', 'red_blue_gap'}
    valid_races     = {'hisp', 'black', 'white'}
    
    check(election, valid_elections)
    check(metric  , valid_metrics)
    check(races   , valid_races)
    # if all races included, disable constant term to prevent colinearity
    if valid_races.issubset(races):
        const = False

    target = f'{election}_{metric}'
    races = {f'vap_{r}_pct': r for r in races}
    predictors = list(races.keys()) + listify(predictors)
    cols = [target, weight] + predictors
    if const:
        predictors.append('const')
    print(f'target column = {target}\nweight column = {weight}\npredictors = {predictors}')
    
    # get columns we need and rename for convenience
    X = df[cols].rename(columns=races)
    
    # create interaction columns
    for a, b in listify(interactions):
        X[a+b] = X[a] * X[b]
    
    # create constant column
    if const:
        X['const'] = 100.0

    # drop rows with missing values - typically small vtds with no recorded votes in this election
    X.dropna(inplace=True)
    
    # pop the target and weight columns from X
    y = X.pop(target)
    w = X.pop(weight)
    
    # create WLS model
    mod = sm.WLS(y, X, w)
    res = mod.fit()
    print(res.summary())
    return df, mod, res

In [ ]:
###### Generate Raw Data - Users cannot run this unless they have access to the source table in BigQuery ######

%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
src_tbl = f'{root_bq}.TX_sldl_planh2100.cntyvtd_0_nodes'
src_cols = get_cols(src_tbl)

def get_vra(level):
    if level == 'county':
        labels = 'county'
    elif level == 'cntyvtd':
        labels = 'cntyvtd, county'
    else:
        raise Exception(f'invalid level {level}')

    sels = []
    votes = []
    diff = []
    red = []
    def f(a, b):
        short = dict()
        for p in ['D', 'R']:
            col = [x for x in src_cols if f'{a}_{p}' in x].pop()
            nm = col.split('_')[3]
            short[p] = f'{b}_{nm}'
            sels.append(f'cast(sum({col}) as int) as {short[p]}')
        votes.append(f'{short["D"]} + {short["R"]} as {b}_votes')
        diff .append(f'case when {b}_votes > 0 then ({short["R"]} - {short["D"]}) / {b}_votes * 100 else Null end as {b}_red_blue_gap')
        red  .append(f'case when {b}_votes > 0 then  {short["R"]} / {b}_votes * 100 else Null end as {b}_red_pct')

    for yr in [2020, 2016, 2012]:
        a = f'President_{yr}'
        b = f'Pres_{yr%100}'
        f(a, b)

    for yr in [2020, 2018, 2014, 2012]:
        a = f'USSen_{yr}'
        b = f'Sen_{yr%100}'
        f(a, b)

    query = []
    query.append(f"""
select
    {labels},
    {join_str().join(sels)},
    sum(vap_hisp) as vap_hisp,
    {' + '.join([f'sum({x})' for x in src_cols if 'vap_nonhisp' in x and 'black' in x])} as vap_black,
    {' + '.join([f'sum({x})' for x in src_cols if 'vap_nonhisp' in x and 'white' in x and 'black' not in x])} as vap_white,
    sum(aland) as aland,
    st_union_agg(polygon) as polygon,
from
    {src_tbl}
group by
    {labels}
""")
    
    query.append(f"""
select
    *,
    vap_hisp + vap_black + vap_white as vap_pop,
    {join_str().join(votes)},
from (
    {subquery(query[-1])}
    )
""")

    query.append(f"""
select
    {labels},
    vap_pop,
    vap_pop / aland as density,
    vap_hisp  / vap_pop * 100 as vap_hisp_pct,
    vap_black / vap_pop * 100 as vap_black_pct,
    vap_white / vap_pop * 100 as vap_white_pct,
    st_distance(polygon, (select polygon from {data_bq}.countries where country = 'Mexico')) / {m_per_mi} as dist_border,
    {join_str().join(red)},
    {join_str().join(diff)},
    vap_hisp,
    vap_black,
    vap_white,
    * except ({labels}, vap_hisp, vap_black, vap_white, vap_pop, aland, polygon),
    aland,
    polygon,
from (
    {subquery(query[-1])}
    )
where vap_pop > 0
""")
    return query[-1]
    
for level in ['county', 'cntyvtd']:
    print(level)
    query = get_vra(level)
    targ_tbl = f'{root_bq}.VRA.{level}'
    load_table(tbl=targ_tbl, query=query)
    df = run_query(f'select * except (polygon) from {targ_tbl}')
    f = data_path / f'vra_{level}.csv'
    f.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(f, index=False)

In [ ]:
df = hisp_redness(election='Sen_18')
# 42.953, 88.548
# df.to_csv(f'{path}/VRA_min_hisp_red_pct_{level}.csv')
df.head(3)

In [ ]:
165/408
430-243

In [ ]:
###### Specify model options and call function above to generate it ######
###### You may create many copies of this cell to try different model configurations ######
opts = {'level'       : 'cntyvtd',
        'metric'      : 'red_blue_gap',
        'election'    : 'Pres_20',
        'const'       : True,
        'races'       : [
            'hisp',
            'black',
            'white',
        ],
        'predictors'  : [
            'density',
            'dist_border',
        ],
        'interactions': [
            ['hisp','black'],
        ],
       }
df, mod, res = model_votes(**opts)

In [ ]:
###### Specify model options and call function above to generate it ######
###### You may create many copies of this cell to try different model configurations ######
opts = {'level'       : 'cntyvtd',
        'metric'      : 'red_blue_gap',
        'election'    : 'Pres_20',
        'const'       : True,
        'races'       : [
            'hisp',
            'black',
            # 'white',
        ],
        'predictors'  : [
            'density',
            'dist_border',
        ],
        'interactions': [
            ['hisp','black'],
        ],
       }
df, mod, res = model_votes(**opts)

In [ ]:
###### Generate Raw Data - Users cannot run this unless they have access to the source table in BigQuery ######

%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
src_tbl = f'{root_bq}.TX_sldl_planh2100.cntyvtd_0_nodes'
src_cols = get_cols(src_tbl)

def get_vra(level):
    if level == 'county':
        labels = 'county'
    elif level == 'cntyvtd':
        labels = 'cntyvtd, county'
    else:
        raise Exception(f'invalid level {level}')

    sels = []
    votes = []
    diff = []
    red = []
    def f(a, b):
        short = dict()
        for party, color in {'D':'blue', 'R':'red'}.items():
            col = [x for x in src_cols if f'{a}_{party}' in x].pop()
            nm = col.split('_')[3]
            short[party] = f'{b}_{color}_votes_{nm}'
            sels.append(f'cast(sum({col}) as int) as {short[party]}')
        votes.append(f'{short["D"]} + {short["R"]} as {b}_votes')
        diff.append(f'case when {b}_votes > 0 then ({short["R"]} - {short["D"]}) / {b}_votes * 100 else Null end as {b}_red_blue_gap')
        red .append(f'case when {b}_votes > 0 then  {short["R"]} / {b}_votes * 100 else Null end as {b}_red_pct')

    for yr in [2020, 2016, 2012]:
        a = f'President_{yr}'
        b = f'Pres_{yr%100}'
        f(a, b)

    for yr in [2020, 2018, 2014, 2012]:
        a = f'USSen_{yr}'
        b = f'Sen_{yr%100}'
        f(a, b)

    query = []
    query.append(f"""
select
    {labels},
    {join_str().join(sels)},
    sum(vap_hisp) as vap_hisp,
    {' + '.join([f'sum({x})' for x in src_cols if 'vap_nonhisp' in x and 'black' in x])} as vap_black,
    {' + '.join([f'sum({x})' for x in src_cols if 'vap_nonhisp' in x and 'white' in x and 'black' not in x])} as vap_white,
    sum(aland) as aland,
    st_union_agg(polygon) as polygon,
from
    {src_tbl}
group by
    {labels}
""")
    
    query.append(f"""
select
    *,
    vap_hisp + vap_black + vap_white as vap_pop,
    {join_str().join(votes)},
from (
    {subquery(query[-1])}
    )
""")

    query.append(f"""
select
    {labels},
    vap_pop,
    vap_pop / aland as density,
    vap_hisp  / vap_pop * 100 as vap_hisp_pct,
    vap_black / vap_pop * 100 as vap_black_pct,
    vap_white / vap_pop * 100 as vap_white_pct,
    st_distance(polygon, (select polygon from {data_bq}.countries where country = 'Mexico')) / {m_per_mi} as dist_border,
    {join_str().join(red)},
    {join_str().join(diff)},
    vap_hisp,
    vap_black,
    vap_white,
    * except ({labels}, vap_hisp, vap_black, vap_white, vap_pop, aland, polygon),
    aland,
    polygon,
from (
    {subquery(query[-1])}
    )
where vap_pop > 0
""")
    return query[-1]
    
for level in ['county', 'cntyvtd']:
    print(level)
    query = get_vra(level)
    targ_tbl = f'{root_bq}.VRA.{level}'
    load_table(tbl=targ_tbl, query=query)
    # df = run_query(f'select * except (polygon) from {targ_tbl}')
    df = run_query(f'select * from {targ_tbl}')
    f = data_path / f'vra_{level}.csv'
    f.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(f, index=False)